# Visualising results for Groene Mient inspired energy community

This notebook visualises the results of experiments performed on groene mient inspired energy community

In [5]:
# importing required libraries
from experiments.experiment import *
from results.results import *
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import ast
from tqdm import tqdm

import warnings

warnings.filterwarnings("ignore")

## 1. Read the results

In [6]:
# read the results

ValueError: invalid literal for int() with base 10: ''

In [11]:
folder = './output/'
ec_name = "GroenMient"
folder = folder + ec_name + '/'
mypath = os.getcwd() + '/output/' + ec_name + '/'
outputs = [f for f in listdir(mypath) if isfile(join(mypath, f))]

all_results = {}

for condition_output in outputs:
    path = f'{folder}{condition_output}'
    df = pd.read_csv(path)

    condition_idx = int(condition_output[22:-4])  # Takes only the number of the condition
    all_results[condition_idx] = df

In [13]:
results = all_results

## 2. Prepare the results

In [14]:
# input details of experiments
number_of_steps = 365
number_of_simulation_runs = 10

x_time = np.arange(0, number_of_simulation_runs * number_of_steps)
for i in range(len(x_time)):
    x_time[i] = x_time[i] % number_of_steps

for experiment, runs in results.items():
    runs.index = x_time

In [15]:
# Get levers details and experiment setup
same_levers, unique_levers = get_unique_levers_dict()

setting up the experiments...



In [16]:
# Results to be printed
for experiment, runs in results.items():
    column_names = list(runs.columns)[1:]
    break

column_names = column_names[1:]
print(column_names)

['M1: realised_demand', 'M2: scheduled_demand', 'M3: shifted_load', 'M4: total_generation', 'M5: savings_on_ToD', 'M6: energy_costs']


In [18]:
# separate results for each experiment run
separated_results = {}
outlier_price_plastics = {}

for experiment, runs in results.items():
    separated_results[experiment] = []
    outlier_price_plastics[experiment] = []
    for run in range(number_of_simulation_runs):
        run_df = runs.iloc[:number_of_steps, :]
        runs = runs.iloc[number_of_steps + 1:, :]

        separated_results[experiment].append(run_df)

## 3. Plot scheduled demand vs realised demand (residential)

In [19]:
rows = 7
columns = 4
fig, axs = plt.subplots(rows, columns, sharey=True, sharex=True, figsize=(20, 16))
fig.suptitle('Scheduled demand vs realised demand for residential community members', fontsize=20, y=0.93)
# fig.text(0.5, 0.04, 'Year', ha='center', fontsize=16)
fig.text(0.04, 0.5, 'kWh', va='center', rotation='vertical', fontsize=16)

# Create index of list of indices for axes
axs_index = []
for i in range(rows):  # iterate through the rows of the subplot grid
    for j in range(columns):  # iterate through the columns of the subplot grid
        axs_index.append(axs[i, j])
axs_index = axs_index[:-1]  # remove last axis as it is not needed

# lever combination number is the unique combination of policy scenarios
lever_combination_number = 0

# iterate through the columns and rows of the subplot grid
for ax in tqdm(axs_index):
    corresponding_experiments = same_levers[
        lever_combination_number]  # List of experiment index having same lever combination
    for experiment_index in corresponding_experiments:  # iterate through the experiments having same lever combination
        experiment_results = separated_results[
            experiment_index]  # Get results of experiment with index experiment_index
        for result in experiment_results:  # iterate through the simulation runs of experiment with index experiment_index
            realised_demand = extract_df_from_json(result, 'M1: realised_demand')
            realised_demand['date'] = pd.to_datetime(realised_demand['date'], format='%Y-%m-%d')
            realised_demand.set_index('date', inplace=True, drop=True)
            scheduled_demand = extract_df_from_json(result, 'M2: scheduled_demand')
            scheduled_demand['date'] = pd.to_datetime(scheduled_demand['date'], format='%Y-%m-%d')
            scheduled_demand.set_index('date', inplace=True, drop=True)
            # collect results for residential and non-residential community members separately
            residential = realised_demand.columns.to_list()[:-1]
            non_residential = realised_demand.columns.to_list()[-1:]
            # calculate total results for residential and non-residential community members
            realised_demand['residential'] = realised_demand.loc[:, residential].sum(axis=1)
            scheduled_demand['residential'] = scheduled_demand.loc[:, residential].sum(axis=1)
            # remove unwanted columns
            columns_to_keep = ['residential']
            realised_demand = realised_demand[columns_to_keep]
            scheduled_demand = scheduled_demand[columns_to_keep]

            # set lower cap to zero
            realised_demand[realised_demand<0] = 0
            scheduled_demand[scheduled_demand<0] = 0

            # plot results for residential and non-residential community members
            scheduled_demand[['residential']].plot(ax=ax, color='#6ebbd5', legend=False, linewidth=0.2, alpha=0.3)
            realised_demand[['residential']].plot(ax=ax, colormap='hsv', legend=False, linewidth=0.2, alpha=0.2)
            ax.set_title(f"Lever combination {unique_levers.loc[lever_combination_number, 'lever information']}",
                         fontsize=12)
    lever_combination_number += 1

# show legends
handles, labels = ax.get_legend_handles_labels()
labels = ['scheduled demand', 'realised (actual) demand']
fig.legend(handles[:2], labels[:2], loc=4, bbox_to_anchor=(0.925,0.15))

# remove last axis as it is not needed
axs[6, 3].axis('off')

plt.savefig('figures/groene_mient/scheduled_demand_vs_realised_demand_residential.png')

 78%|███████▊  | 21/27 [9:46:53<2:47:41, 1676.84s/it]

KeyboardInterrupt



Error in callback <function flush_figures at 0x00000221040B8D30> (for post_execute):



KeyboardInterrupt



## 4. Plot scheduled demand vs realised demand (non-residential)

In [ ]:
rows = 7
columns = 4
fig, axs = plt.subplots(rows, columns, sharey=True, sharex=True, figsize=(20, 16))
fig.suptitle('Scheduled demand vs realised demand for non-residential community members', fontsize=20, y=0.93)
# fig.text(0.5, 0.04, 'Year', ha='center', fontsize=16)
fig.text(0.04, 0.5, 'kWh', va='center', rotation='vertical', fontsize=16)

# Create index of list of indices for axes
axs_index = []
for i in range(rows):  # iterate through the rows of the subplot grid
    for j in range(columns):  # iterate through the columns of the subplot grid
        axs_index.append(axs[i, j])
axs_index = axs_index[:-1]  # remove last axis as it is not needed

# lever combination number is the unique combination of policy scenarios
lever_combination_number = 0

# iterate through the columns and rows of the subplot grid
for ax in tqdm(axs_index):
    corresponding_experiments = same_levers[
        lever_combination_number]  # List of experiment index having same lever combination
    for experiment_index in corresponding_experiments:  # iterate through the experiments having same lever combination
        experiment_results = separated_results[
            experiment_index]  # Get results of experiment with index experiment_index
        for result in experiment_results:  # iterate through the simulation runs of experiment with index experiment_index
            realised_demand = extract_df_from_json(result, 'M1: realised_demand')
            realised_demand['date'] = pd.to_datetime(realised_demand['date'], format='%Y-%m-%d')
            realised_demand.set_index('date', inplace=True, drop=True)
            scheduled_demand = extract_df_from_json(result, 'M2: scheduled_demand')
            scheduled_demand['date'] = pd.to_datetime(scheduled_demand['date'], format='%Y-%m-%d')
            scheduled_demand.set_index('date', inplace=True, drop=True)
            # collect results for residential and non-residential community members separately
            residential = realised_demand.columns.to_list()[:-1]
            non_residential = realised_demand.columns.to_list()[-1:]
            # calculate total results for residential and non-residential community members
            realised_demand['non_residential'] = realised_demand.loc[:, non_residential].sum(axis=1)
            scheduled_demand['non_residential'] = scheduled_demand.loc[:, non_residential].sum(axis=1)
            # remove unwanted columns
            columns_to_keep = ['non_residential']
            realised_demand = realised_demand[columns_to_keep]
            scheduled_demand = scheduled_demand[columns_to_keep]

            # set lower cap to zero
            realised_demand[realised_demand<0] = 0
            scheduled_demand[scheduled_demand<0] = 0

            # plot results for residential and non-residential community members
            scheduled_demand[['non_residential']].plot(ax=ax, color='#6ebbd5', legend=False, linewidth=0.2, alpha=0.1)
            realised_demand[['non_residential']].plot(ax=ax, colormap='hsv', legend=False, linewidth=0.2, alpha=0.5)
            ax.set_title(f"Lever combination {unique_levers.loc[lever_combination_number, 'lever information']}",
                         fontsize=12)
    lever_combination_number += 1

# show legends
handles, labels = ax.get_legend_handles_labels()
labels = ['scheduled demand', 'realised (actual) demand']
fig.legend(handles[:2], labels[:2], loc=4, bbox_to_anchor=(0.925,0.15))

# remove last axis as it is not needed
axs[6, 3].axis('off')

plt.savefig('figures/groene_mient/scheduled_demand_vs_realised_demand_nonresidential.png')

## 5. Plot shifted demand through demand response

In [ ]:
rows = 7
columns = 4
fig, axs = plt.subplots(rows, columns, sharey=True, sharex=True, figsize=(20, 16))
fig.suptitle('Shifted load through demand response', fontsize=20, y=0.93)
# fig.text(0.5, 0.04, 'Year', ha='center', fontsize=16)
fig.text(0.04, 0.5, 'kWh', va='center', rotation='vertical', fontsize=16)

# Create index of list of indices for axes
axs_index = []
for i in range(rows):  # iterate through the rows of the subplot grid
    for j in range(columns):  # iterate through the columns of the subplot grid
        axs_index.append(axs[i, j])
axs_index = axs_index[:-1]  # remove last axis as it is not needed

# lever combination number is the unique combination of policy scenarios
lever_combination_number = 0

# iterate through the columns and rows of the subplot grid
for ax in tqdm(axs_index):
    corresponding_experiments = same_levers[
        lever_combination_number]  # List of experiment index having same lever combination
    for experiment_index in corresponding_experiments:  # iterate through the experiments having same lever combination
        experiment_results = separated_results[
            experiment_index]  # Get results of experiment with index experiment_index
        for result in experiment_results:  # iterate through the simulation runs of experiment with index experiment_index
            shifted_load = extract_df_from_json(result, 'M3: shifted_load')
            shifted_load['date'] = pd.to_datetime(shifted_load['date'], format='%Y-%m-%d')
            shifted_load.set_index('date', inplace=True, drop=True)
            # collect results for residential and non-residential community members separately
            residential = shifted_load.columns.to_list()[:-1]
            non_residential = shifted_load.columns.to_list()[-1:]
            # calculate total results for residential and non-residential community members
            shifted_load['residential'] = shifted_load.loc[:, residential].sum(axis=1)
            shifted_load['non_residential'] = shifted_load.loc[:, non_residential].sum(axis=1)
            # remove unwanted columns
            columns_to_keep = ['residential', 'non_residential']
            shifted_load = shifted_load[columns_to_keep]
            shifted_load[shifted_load>4000] = 4000

            # plot results for community members
            shifted_load.plot(ax=ax, colormap='tab10', legend=False, linewidth=0.2, alpha=0.5)
            ax.set_title(f"Lever combination {unique_levers.loc[lever_combination_number, 'lever information']}",
                         fontsize=12)
    lever_combination_number += 1


# show legends
handles, labels = ax.get_legend_handles_labels()
labels = ['load shifted as demand response by residential members', 'load shifted as demand response by non-residential members']
fig.legend(handles[:2], labels[:2], loc=4, bbox_to_anchor=(0.925,0.15))

# remove last axis as it is not needed
axs[6, 3].axis('off')

plt.savefig('figures/greone_mient/shifted_load.png', dpi=300)

## 6. Plot energy costs vs savings

In [ ]:
rows = 7
columns = 4
fig, axs = plt.subplots(rows, columns, sharey=True, sharex=True, figsize=(20, 16))
fig.suptitle('Expenditure and savings on electricity through demand response', fontsize=20, y=0.93)
# fig.text(0.5, 0.04, 'Year', ha='center', fontsize=16)
fig.text(0.04, 0.5, 'Euros', va='center', rotation='vertical', fontsize=16)

# Create index of list of indices for axes
axs_index = []
for i in range(rows):  # iterate through the rows of the subplot grid
    for j in range(columns):  # iterate through the columns of the subplot grid
        axs_index.append(axs[i, j])
axs_index = axs_index[:-1]  # remove last axis as it is not needed

# lever combination number is the unique combination of policy scenarios
lever_combination_number = 0

# iterate through the columns and rows of the subplot grid
for ax in tqdm(axs_index):
    corresponding_experiments = same_levers[
        lever_combination_number]  # List of experiment index having same lever combination
    for experiment_index in corresponding_experiments:  # iterate through the experiments having same lever combination
        experiment_results = separated_results[
            experiment_index]  # Get results of experiment with index experiment_index
        for result in experiment_results:  # iterate through the simulation runs of experiment with index experiment_index

            savings = extract_df_from_json(result, 'M5: savings_on_ToD')
            savings['date'] = pd.to_datetime(savings['date'], format='%Y-%m-%d')
            savings.set_index('date', inplace=True, drop=True)

            costs = extract_df_from_json(result, 'M6: energy_costs')
            costs['date'] = pd.to_datetime(costs['date'], format='%Y-%m-%d')
            costs.set_index('date', inplace=True, drop=True)

            # collect results for residential and non-residential community members separately
            residential = savings.columns.to_list()[:-1]
            non_residential = savings.columns.to_list()[-1:]

            # calculate total results for residential and non-residential community members
            costs['costs'] = costs.sum(axis=1)
            costs['savings'] = savings.sum(axis=1)

            columns_to_keep = ['costs', 'savings']
            costs = costs[columns_to_keep]

            #set lower cap to zero
            costs[costs<0] = 0
            costs.loc[costs['savings']>2000, 'savings'] = 2100

            # plot results
            costs.plot(ax=ax,colormap = 'tab20b', legend=False, linewidth=0.2, alpha=0.2)
            ax.set_title(f"Lever combination {unique_levers.loc[lever_combination_number, 'lever information']}",
                         fontsize=12)
    lever_combination_number += 1

# show legends
handles, labels = ax.get_legend_handles_labels()
labels = ['cost of importing electricity from grid', 'savings on electricity cost through demand response']
fig.legend(handles[:2], labels[:2],loc=4, bbox_to_anchor=(0.925,0.15))

# remove last axis as it is not needed
axs[6, 3].axis('off')

plt.savefig('figures/greone_mient/costs.png', dpi=300)

## 7. Plot community overview

In [ ]:
rows = 7
columns = 4
fig, axs = plt.subplots(rows, columns, sharey=True, sharex=True, figsize=(20, 16))
fig.suptitle('Electricity generation and consumption for the community', fontsize=20, y=0.93)
fig.text(0.04, 0.5, 'kWh', va='center', rotation='vertical', fontsize=16)

# Create index of list of indices for axes
axs_index = []
for i in range(rows):  # iterate through the rows of the subplot grid
    for j in range(columns):  # iterate through the columns of the subplot grid
        axs_index.append(axs[i, j])
axs_index = axs_index[:-1]  # remove last axis as it is not needed

# lever combination number is the unique combination of policy scenarios
lever_combination_number = 0

# iterate through the columns and rows of the subplot grid
for ax in tqdm(axs_index):
    corresponding_experiments = same_levers[
        lever_combination_number]  # List of experiment index having same lever combination
    for experiment_index in corresponding_experiments:  # iterate through the experiments having same lever combination
        experiment_results = separated_results[
            experiment_index]  # Get results of experiment with index experiment_index
        for result in experiment_results:  # iterate through the simulation runs of experiment with index experiment_index

            realised_demand = extract_df_from_json(result, 'M1: realised_demand')
            realised_demand['date'] = pd.to_datetime(realised_demand['date'], format='%Y-%m-%d')
            realised_demand.set_index('date', inplace=True, drop=True)

            generation = extract_df_from_json(result, 'M4: total_generation')
            generation['date'] = pd.to_datetime(generation['date'], format='%Y-%m-%d')
            generation.set_index('date', inplace=True, drop=True)

            shifted_load = extract_df_from_json(result, 'M3: shifted_load')
            shifted_load['date'] = pd.to_datetime(shifted_load['date'], format='%Y-%m-%d')
            shifted_load.set_index('date', inplace=True, drop=True)

            # calculate total results for residential and non-residential community members
            realised_demand['residential'] = realised_demand.loc[:, residential].sum(axis=1)
            realised_demand['non_residential'] = realised_demand.loc[:, non_residential].sum(axis=1)

            shifted_load['total_shifted_load'] = shifted_load.loc[:, residential].sum(axis=1)
            shifted_load = shifted_load['total_shifted_load']

            columns_to_keep = ['residential', 'non_residential']
            realised_demand = realised_demand[columns_to_keep]

            # set lower cap to zero
            realised_demand[realised_demand<0] = 0
            shifted_load[shifted_load>4000] = 4000

            # plot results for community members
            realised_demand.plot(ax=ax, colormap='Pastel2', kind="area", stacked=True, legend=False, linewidth=0.2,
             alpha=0.5)
            generation.plot(ax=ax, colormap = 'tab10', legend=False, linewidth=0.2, alpha=0.25)
            shifted_load.plot(ax=ax, colormap = 'Set1', legend=False, linewidth=0.2, alpha=0.5)
            ax.set_title(f"Lever combination {unique_levers.loc[lever_combination_number, 'lever information']}",
                         fontsize=12)


    lever_combination_number += 1

# show legends
handles, labels = ax.get_legend_handles_labels()
labels = ['electricity consumption of residential members', 'electricity consumption of non-residential members', 'total generation from community assets', 'total shifted demand through demand response']
fig.legend(handles[:4], labels[:4], loc=4, bbox_to_anchor=(0.925,0.15))

# remove last axis as it is not needed
axs[6, 3].axis('off')

plt.savefig('figures/greone_mient/community_overview.png', dpi=300)